In [1]:
from bs4 import BeautifulSoup
import requests
# import pandas as pd
import time
import re
import urllib
import os

In [2]:
DATAFOLDER = '../data/irasutoya/'
print(os.listdir(DATAFOLDER))

['bicycle_ichirinsya_boy.png', 'bicycle_ichirinsya_girl.png', 'christian_altar_boy_candle.png', 'christian_altar_boy_corss.png', 'christian_altar_girl_candle.png', 'christian_altar_girl_cross.png', 'fusen_hanasu_boy.png', 'gassyuku_shokuji_boys.png', 'gassyuku_shokuji_girls.png', 'gassyuku_sleep_boys.png', 'gassyuku_sleep_girls.png', 'kids_mokkou_kyoushitsu_boy.png', 'kids_mokkou_kyoushitsu_girl.png', 'kids_school_mokuiku_tsumiki.png', 'kid_kigae_oyako_father.png', 'kid_kigae_oyako_mother.png', 'kid_seikaku_uchiki_boy.png', 'kid_seikaku_uchiki_girl.png', 'kid_toy_neji.png', 'kouen_untei_boy.png', 'kurumaisu_reclining_boy.png', 'medical_kyuunyuuki_spacer_boy.png', 'medical_kyuunyuuki_spacer_girl.png', 'medical_xray_rentogen_boy.png', 'medical_xray_rentogen_girl.png', 'message_suwaranaidene.png', 'mother_nagusameru_boy.png', 'music_gassyou_kids_asia.png', 'music_gassyou_kids_asia_formal.png', 'music_gassyou_kids_black.png', 'music_gassyou_kids_black_formal.png', 'music_gassyou_kids_india

In [3]:
url = 'https://www.irasutoya.com/2020/01/blog-post_335.html'

In [4]:
def get_categories(url):
    soup = BeautifulSoup(requests.get(url).content)
    cat_url_list = []
    cat_name_list = []
    href_list = soup.find('div', {'class':'widget-content list-label-widget-content'}).find_all('a')
    for href in href_list:
        cat_url_list.append(href['href'])
        cat_name_list.append(href.text)
    return cat_url_list, cat_name_list

In [5]:
def get_image_urls(cat_url):
    soup = BeautifulSoup(requests.get(cat_url).content)
    img_url_list = []
    img_name_list = []
    img_href_list = soup.find_all('div', {'class':"boxmeta clearfix"})
    N = 0
    while(N<300):
        for href in img_href_list:
            img_url_list.append(href.find('a')['href'])
            img_name_list.append(href.find('a').text)
        try:
            next_url = soup.find('div', {'class':'blog-pager'}).find('a', {'title':"前の投稿"})['href']
        except:
            break
        if next_url == None:
            break
        soup = BeautifulSoup(requests.get(next_url).content)
        img_href_list = soup.find_all('div', {'class':"boxmeta clearfix"})
        N += 1
    return img_url_list, img_name_list

In [6]:
def get_image(img_url, cat):
    soup = BeautifulSoup(requests.get(img_url).content)
    images = soup.find_all('div', {'class':"separator"})
    if len(images) == 0:
        return 0
    caption = images[-1].text
    out = []
    for obj in images:
        obj_row = obj.find_all('a')
        for ob in obj_row:
            try:
                img_url = ob.find('img')['src']
                img_title = ob.find('img')['alt']
                img_name = img_url.split('/')[-1]
                out.append((img_title, img_name, img_url, caption, cat))
                path = DATAFOLDER + img_name

                data = urllib.request.urlopen(img_url).read()
                with open(path, mode="wb") as f:
                    f.write(data)
            except:
                break
#     print(caption)
    return out

In [7]:
import sqlite3
conn = sqlite3.connect('irasutoya.db')
c = conn.cursor()
c.execute("DROP TABLE IF EXISTS irasuto")
c.execute('create table irasuto(title text, name text, url text, caption text, category text)')

In [8]:
url = 'https://www.irasutoya.com/2020/01/blog-post_335.html'
cat_url_list, cat_name_list = get_categories(url)
num = len(cat_name_list)
cnt = 0
for cat_url, cat_name in zip(cat_url_list[1:], cat_name_list[1:]):
    start = time.time()
    print(cat_name, ':', cnt, "/", num)
    img_url_list, img_name_list = get_image_urls(cat_url)
    for img_url in img_url_list:
        out = get_image(img_url, cat_name)
        try:
            c.executemany("INSERT INTO irasuto VALUES (?, ?, ?, ?, ?)", out)
        except:
            continue
        time.sleep(1)
    end = time.time()
    cnt += 1
    minute = (end - start) / 60
    print(minute)

こども : 0 / 240
29.32157874107361
職業 : 1 / 240
29.928056172529857
病気 : 2 / 240
28.36400633653005
ポーズ : 3 / 240
26.661270010471345
会社 : 4 / 240
25.961610162258147
お金 : 5 / 240
26.42564485470454
道具 : 6 / 240
26.572399401664732
違反 : 7 / 240
23.667928206920624
ビジネス : 8 / 240
22.935407483577727
学校 : 9 / 240
22.20919200181961
ファッション : 10 / 240
22.730994713306426
事故 : 11 / 240
22.649257258574167
食べ物 : 12 / 240
23.827578087647755
医療 : 13 / 240
21.718619815508525
趣味 : 14 / 240
22.131373167037964
スポーツ : 15 / 240
19.630711205800374
建物 : 16 / 240
19.61706923643748
スイーツ : 17 / 240
19.68037089506785
旅行 : 18 / 240
17.717637900511424
おもちゃ : 19 / 240
17.777160171667735
家電 : 20 / 240
17.283121017615
家族 : 21 / 240
16.554326903820037
キャラクター : 22 / 240
16.478808347384135
文字 : 23 / 240
17.0192085703214
料理 : 24 / 240
15.894000748793284
動物キャラ : 25 / 240
14.929784182707468
機械 : 26 / 240
15.150954461097717
マーク : 27 / 240
15.908281946182251
日本 : 28 / 240
13.889932664235433
医療機器 : 29 / 240
14.115196907520295
音楽 : 3

MissingSchema: Invalid URL '//www.blogger.com/rearrange?blogID=8749391503645026985&widgetType=Label&widgetId=Label2&action=editWidget&sectionId=main': No schema supplied. Perhaps you meant http:////www.blogger.com/rearrange?blogID=8749391503645026985&widgetType=Label&widgetId=Label2&action=editWidget&sectionId=main?

In [9]:
# c.execute('SELECT * FROM irasuto')
# for row in c.fetchall():
#     print(row)

In [10]:
conn.commit()
conn.close()